In [34]:
# Transformers installation
! pip install transformers datasets

In [35]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving convertedQA.csv to convertedQA (1).csv


In [36]:
import csv
from datasets import Dataset

# 加载你的数据集
def load_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        # 直接返回读取器对象也可以，但为了后续处理方便，这里转换为列表
        dataset = list(reader)
    return dataset


dataset_file = "convertedQA.csv"  # 请将路径替换为你的文件路径
my_dataset = load_dataset(dataset_file)
my_dataset[0]
# 假设CSV文件中有名为'instruction'和'response'的列，我们


{'Instruction': 'What is the main purpose of the EU AI Act?',
 'Response': 'The EU AI Act aims to regulate the ethical use of AI by ensuring AI systems are safe, respect fundamental rights, and foster responsible development.'}

In [37]:
# 创建Dataset对象
dataset = Dataset.from_dict({
    'text': [f"Instruction: {example['Instruction']} Response: {example['Response']}" for example in my_dataset]
})

In [38]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization函数
def tokenize_function(examples):
    # 此处假设examples包含一个'text'字段，它是合并后的文本
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# 应用Tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 接下来，你可以根据需要处理tokenized_dataset
# 例如，随机选择一小部分数据进行实验
train_dataset = tokenized_dataset.shuffle(seed=42).select(range(100))
eval_dataset = tokenized_dataset.shuffle(seed=42).select(range(100))

Map:   0%|          | 0/1023 [00:00<?, ? examples/s]

In [39]:
# Load model directly
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-large-uncased")

Some weights of the model checkpoint at google-bert/bert-large-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
# !pip install transformers[torch]

In [41]:
# !pip install accelerate -U

In [42]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [43]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [44]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer.train()